In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
import cv2
from random import shuffle
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


In [2]:
from glob import glob
data = list(glob('COVID/*.png'))
print(len(data))
datapng = list(glob('COVID/*.jpg'))
print(len(datapng))
datajpeg = list(glob('COVID/*.jpeg'))
print(len(datajpeg))


514
1796
1734


In [3]:
img_rows = 128
img_cols = 128
images = []
for i in datajpeg:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images.append(resized_img)
for i in datapng:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images.append(resized_img)
for i in data:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images.append(resized_img)


In [4]:
images = np.array(images)/255.0
images = np.reshape(images, (images.shape[0], img_rows, img_cols, 3))


In [5]:
len(images)


4044

In [6]:
datan = list(glob('Non-COVID/*.png'))
print(len(datan))
datapngn = list(glob('Non-COVID/*.jpg'))
print(len(datapngn))
datajpegn = list(glob('Non-COVID/*.jpeg'))
print(len(datajpegn))


1861
955
2677


In [7]:
img_rows = 128
img_cols = 128
images1 = []
for i in datan:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images1.append(resized_img)
for i in datapngn:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images1.append(resized_img)

for i in datajpegn:
    img = cv2.imread(i)
    resized_img = cv2.resize(img, (img_rows, img_cols))
    images1.append(resized_img)


In [8]:
print(len(images1))
images1 = np.array(images1)/255.0
images1 = np.reshape(images1, (images1.shape[0], img_rows, img_cols, 3))


5493


In [9]:
label = np.empty(4044)
label.fill(1)
#1 means covid and 0 means non
label1 = np.empty(5493)
label1.fill(0)


In [10]:
label = np.concatenate((label, label1))
print(label.shape)


(9537,)


In [11]:
images = np.concatenate((images, images1), axis=0)
images.shape


(9537, 128, 128, 3)

In [12]:
imagesfinal = np.concatenate((images[0:2000], images[6000:8000]), axis=0)
imagesfinal.shape


(4000, 128, 128, 3)

In [13]:
labelfinal = np.concatenate((label[0:2000], label[6000:8000]))
labelfinal.shape


(4000,)

In [14]:
lb = LabelBinarizer()
labelfinal = lb.fit_transform(labelfinal)
labelfinal = to_categorical(labelfinal)
labelfinal = np.array(labelfinal)


In [15]:
(X_train, X_test, y_train, y_test) = train_test_split(
    imagesfinal, labelfinal, test_size=0.25, random_state=0)


In [16]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [17]:

base = MobileNetV2(weights="imagenet", include_top=False,
                   input_tensor=Input((128, 128, 3)))


In [18]:
for layer in base.layers:
    layer.trainable=False

In [19]:
model = base.output
model = AveragePooling2D()(model)
model = Flatten(name="flatten")(model)
model = Dense(64, activation="relu")(model)
model = Dropout(0.2)(model)
model = Dense(2, activation="softmax")(model)


In [20]:
final = Model(inputs=base.input, outputs=model)
final.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
final.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [21]:
X_train.shape

(3000, 128, 128, 3)

In [22]:
final.fit(X_train, y_train, epochs=20)


Epoch 1/20


C:\Users\HP\Desktop\python\lib\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


94/94 [==============================] - 44s 396ms/step - loss: 0.3958 - accuracy: 0.8540
Epoch 2/20
94/94 [==============================] - 34s 362ms/step - loss: 0.2273 - accuracy: 0.9213
Epoch 3/20
94/94 [==============================] - 33s 349ms/step - loss: 0.1764 - accuracy: 0.9360
Epoch 4/20
94/94 [==============================] - 33s 352ms/step - loss: 0.1553 - accuracy: 0.9457
Epoch 5/20
94/94 [==============================] - 35s 369ms/step - loss: 0.1307 - accuracy: 0.9500
Epoch 6/20
94/94 [==============================] - 35s 371ms/step - loss: 0.0983 - accuracy: 0.9653
Epoch 7/20
94/94 [==============================] - 36s 381ms/step - loss: 0.1024 - accuracy: 0.9613
Epoch 8/20
94/94 [==============================] - 35s 369ms/step - loss: 0.0790 - accuracy: 0.9713
Epoch 9/20
94/94 [==============================] - 34s 361ms/step - loss: 0.0869 - accuracy: 0.9693
Epoch 10/20
94/94 [==============================] - 34s 360ms/step - loss: 0.0748 - accuracy: 0.9727


In [23]:

predictions = final.predict(X_test)


In [24]:
pred = lb.inverse_transform(predictions)
y_te = lb.inverse_transform(y_test)


In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te, pred)


0.934

In [26]:
final.save(r'C:\Users\HP\Desktop\iml\covid_detection\X-ray\Mobilev2')


C:\Users\HP\Desktop\python\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: C:\Users\HP\Desktop\iml\covid_detection\X-ray\Mobilev2\assets
